In [1]:
import os
print os.getcwd()

/abaghel/projects/rnn-patient-sentiment-paper


In [2]:
import cPickle
import numpy as np

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten, Reshape, Merge
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD, Adadelta
from keras.constraints import unitnorm, maxnorm
from keras.regularizers import l2
from keras.layers.advanced_activations import PReLU
from keras.preprocessing import sequence

from keras.layers import Dense, Dropout, Activation, Embedding
from keras.layers import LSTM, SimpleRNN, GRU

from sklearn.metrics import roc_auc_score

# This import to resolve some errors with tf version on office server
import tensorflow as tf
tf.python.control_flow_ops = tf

Using TensorFlow backend.


#  Simple Preprocessing without word2vec

In [4]:
import csv
import os
import numpy as np
import pandas as pd
import cPickle
from collections import defaultdict
import re


def RepresentsInt(s):
    try: 
        int(s)
        return True
    except ValueError:
        return False

     
def build_data_train_test(train_ratio = 0.8, clean_string=True):
    """
    Loads data and split into train and test sets.
    """
    revs = []
    vocab = defaultdict(float)
    # Pre-process train data set
    for i in xrange(len(text_train)):
        line = text_train[i]
        y = helpfull_train[i]
        y_cat = np.zeros(5)
        y_cat[helpful_cat_train[i]] =1
        rev = []
        rev.append(line.strip())
        if clean_string:
            orig_rev = clean_str(' '.join(rev))
        else:
            orig_rev = ' '.join(rev).lower()
        words = set(orig_rev.split())
        for word in words:
            vocab[word] += 1
        datum  = {'y': y, 
                  'y_cat': y_cat,
                  'text': orig_rev,
                  'num_words': len(orig_rev.split()),
                  'split': int(np.random.rand() < train_ratio)}
        revs.append(datum)
    # Pre-process test data set
    for i in xrange(len(text_test)):
        line = text_test[i]
        y = helpfull_test[i]    
        y_cat = np.zeros(5)
        y_cat[helpful_cat_test[i]] =1
        rev = []
        rev.append(line.strip())
        if clean_string:
            orig_rev = clean_str(' '.join(rev))
        else:
            orig_rev = ' '.join(rev).lower()
        words = set(orig_rev.split())
        for word in words:
            vocab[word] += 1
        datum  = {'y': y, 
                  'y_cat': y_cat,
                  'text': orig_rev,
                  'num_words': len(orig_rev.split()),
                  'split': -1}
        revs.append(datum)      
    return revs, vocab


In [5]:
text=[]
helpfull=[]
helpful_categorical = []
knowledge=[]
staff=[]
ii=0
jj=0
files = [os.path.join("data/", f) for f in os.listdir('data/') if os.path.isfile(os.path.join("data/", f))]
for f in files:
    if f.endswith(".txt"):
        with open(f) as tsv:
            for line in csv.reader(tsv, dialect="excel-tab"):
                ii=ii+1
                if(len(line) > 6 and RepresentsInt(line[3]) and RepresentsInt(line[5]) and RepresentsInt(line[6]) ):
                    jj=jj+1
                    if int(line[5])>=3:
                        helpfull.append(int(1))
                    elif int(line[5])<3:
                        helpfull.append(int(0))
                    text.append(line[2])
                    helpful_categorical.append(int(line[5])-1)
                    #knowledge.append(int(line[6])-1)
                    #staff.append(int(line[3])-1)
    
        
        
print len(helpfull)
print len(helpful_categorical)
print len(text)
text_train=text[:34000]
text_test=text[34000:]

helpfull_train=helpfull[:34000]
helpfull_test=helpfull[34000:]

helpful_cat_train=helpful_categorical[:34000]
helpful_cat_test=helpful_categorical[34000:]


38200
38200
38200


In [6]:
print helpful_categorical[0:15]

[4, 4, 4, 4, 4, 4, 4, 0, 4, 4, 4, 4, 4, 0, 4]


In [7]:
def get_W(word_vecs, k=300):
    """
    Get word matrix. W[i] is the vector for word indexed by i
    """
    vocab_size = len(word_vecs)
    word_idx_map = dict()
    W = np.zeros(shape=(vocab_size+1, k), dtype=np.float32)
    W[0] = np.zeros(k, dtype=np.float32)
    i = 1
    for word in word_vecs:
        W[i] = word_vecs[word]
        word_idx_map[word] = i
        i += 1
    return W, word_idx_map

def load_bin_vec(fname, vocab):
    """
    Loads 300x1 word vecs from Google (Mikolov) word2vec
    """
    word_vecs = {}
    with open(fname, 'rb') as f:
        header = f.readline()
        print header
        vocab_size, layer1_size = map(int, header.split())
        binary_len = np.dtype('float32').itemsize * layer1_size
        print vocab_size*2/3
        for line in xrange(int(vocab_size*2.5/3)):
            #print line
            word = []
            while True:
                ch = f.read(1)
                if ch == ' ':
                    word = ''.join(word)
                    break
                if ch != '\n':
                    word.append(ch)   
            if word in vocab:
                word_vecs[word] = np.fromstring(f.read(binary_len), dtype='float32')  
            else:
                f.read(binary_len)
            if (line%100000==0):
                print line
    return word_vecs

def add_unknown_words(word_vecs, vocab, min_df=1, k=300):
    """
    For words that occur in at least min_df documents, create a separate word vector.    
    0.25 is chosen so the unknown vectors have (approximately) same variance as pre-trained ones
    """
    for word in vocab:
        if word not in word_vecs and vocab[word] >= min_df:
            word_vecs[word] = np.random.uniform(-0.25,0.25,k)  

def clean_str(string):
    """
    Tokenization/string cleaning for dataset
    Every dataset is lower cased except
    """
    string = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", string)     
    string = re.sub(r"\'s", " \'s", string) 
    string = re.sub(r"\'ve", " \'ve", string) 
    string = re.sub(r"n\'t", " n\'t", string) 
    string = re.sub(r"\'re", " \'re", string) 
    string = re.sub(r"\'d", " \'d", string) 
    string = re.sub(r"\'ll", " \'ll", string) 
    string = re.sub(r",", " , ", string) 
    string = re.sub(r"!", " ! ", string) 
    string = re.sub(r"\(", " \( ", string) 
    string = re.sub(r"\)", " \) ", string) 
    string = re.sub(r"\?", " \? ", string) 
    string = re.sub(r"\s{2,}", " ", string)    
    return string.strip().lower()

In [56]:
revs, vocab = build_data_train_test(train_ratio=0.8, clean_string=True)
max_l = np.max(pd.DataFrame(revs)['num_words'])
print 'data loaded!'
print 'number of sentences: ' + str(len(revs))
print 'vocab size: ' + str(len(vocab))
print 'max sentence length: ' + str(max_l)

data loaded!
number of sentences: 38200
vocab size: 36883
max sentence length: 247


In [57]:
w2v = {}

In [58]:
add_unknown_words(w2v, vocab)
W, word_idx_map = get_W(w2v)

In [11]:
print (len(W))
print (len(vocab))
print (len(word_idx_map.items()))
print "\n\nSome words in the vocab:"
for i in range(10):
    print word_idx_map.keys()[i]

36884
36883
36883


Some words in the vocab:
blgh
pasnecker
colonoscopy
sonja
gag
woods
dogears
hanging
woody
localized


# Prepare for training

In [12]:
def get_idx_from_sent(sent, word_idx_map, max_l=51, kernel_size=5):
    """
    Transforms sentence into a list of indices. Pad with zeroes.
    """
    x = []
    words = sent.split()
    for word in words:
        if word in word_idx_map:
            x.append(word_idx_map[word])
#     while len(x) < max_l:
#         x.append(0)
    return x

def make_idx_data(revs, word_idx_map, max_l=51, kernel_size=5):
    """
    Transforms sentences into a 2-d matrix.
    """
    train, val, test = [], [], []
    y_train, y_val, y_test = [], [] , []
    y_cat_train, y_cat_val, y_cat_test = [], [] , []
    for rev in revs:
        sent = get_idx_from_sent(rev['text'], word_idx_map, max_l, kernel_size)
#         sent.append(rev['y'])
        if rev['split'] == 1:
            train.append(sent)
            y_train.append(int(rev["y"]))
            y_cat_train.append(rev["y_cat"])
        elif rev['split'] == 0:
            val.append(sent)
            y_val.append(int(rev["y"]))
            y_cat_val.append(rev["y_cat"])
        else:
            test.append(sent)
            y_test.append(int(rev["y"]))
            y_cat_test.append(rev["y_cat"])
            
#     train = np.array(train, dtype=np.int)
#     val = np.array(val, dtype=np.int)
#     test = np.array(test, dtype=np.int)
    train = sequence.pad_sequences(train, maxlen=max_l)
    y_train = np.array(y_train, dtype=np.int)
    y_cat_train = np.array(y_cat_train, dtype=np.int)
    
    val = sequence.pad_sequences(val, maxlen=max_l)
    y_val = np.array(y_val, dtype=np.int)
    y_cat_val = np.array(y_cat_val, dtype=np.int)
    
    test = sequence.pad_sequences(test, maxlen=max_l)
    y_test = np.array(y_test, dtype=np.int)
    y_cat_test = np.array(y_cat_test, dtype=np.int)
    
    return [train, val, test, y_train, y_val, y_test, y_cat_train, y_cat_val, y_cat_test]

In [13]:
datasets = make_idx_data(revs, word_idx_map, max_l=200,kernel_size=5)

In [14]:
train, y_train, y_cat_train = datasets[0], datasets[3], datasets[6]
val, y_val, y_cat_val = datasets[1], datasets[4], datasets[7]
test, y_test, y_cat_test = datasets[2], datasets[5], datasets[8]

In [15]:
print len(train), train.shape, len(y_train), y_train.shape, y_train.mean()
print np.unique(y_train)

print np.unique(y_cat_train)
print np.sum(y_cat_train, axis=0)*100/float(np.sum(y_cat_train))

27235 (27235, 200) 27235 (27235,) 0.70086286029
[0 1]
[0 1]
[ 23.64237195   6.27134202   4.02423352   5.15145952  60.91059299]


In [16]:
print len(val), val.shape, len(y_val), y_val.shape, y_val.mean()
print np.unique(y_val)

print np.sum(y_cat_val, axis=0)*100/float(np.sum(y_cat_val))

6765 (6765, 200) 6765 (6765,) 0.70140428677
[0 1]
[ 23.0007391    6.85883222   4.05025868   5.07021434  61.01995565]


In [17]:
print len(test), test.shape, len(y_test), y_test.shape, y_test.mean()
print np.unique(y_test)

print np.sum(y_cat_test, axis=0)*100/float(np.sum(y_cat_test))

4200 (4200, 200) 4200 (4200,) 0.700476190476
[0 1]
[ 24.23809524   5.71428571   3.5          4.57142857  61.97619048]


# Try the model same as IMDB

In [66]:
print('Build model...')
model = Sequential()
model.add(Embedding(len(vocab)+1, 128, dropout=0.2, mask_zero=True))
model.add(LSTM(128, dropout_W=0.2, dropout_U=0.2))  # try using a GRU instead, for fun
model.add(Dense(1))
model.add(Activation('sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Build model...


In [67]:
print('Train...')
model.fit(train, y_train, batch_size=50, nb_epoch=15,validation_data=(val, y_val))

Train...
Train on 27234 samples, validate on 6766 samples
Epoch 1/15
27234/27234 [==============================] - 435s - loss: 0.3745 - acc: 0.8288 - val_loss: 0.1986 - val_acc: 0.9138
Epoch 2/15
27234/27234 [==============================] - 432s - loss: 0.2382 - acc: 0.8960 - val_loss: 0.1803 - val_acc: 0.9228
Epoch 3/15
27234/27234 [==============================] - 430s - loss: 0.1947 - acc: 0.9166 - val_loss: 0.1784 - val_acc: 0.9261
Epoch 4/15
27234/27234 [==============================] - 431s - loss: 0.1652 - acc: 0.9286 - val_loss: 0.1705 - val_acc: 0.9291
Epoch 5/15
27234/27234 [==============================] - 432s - loss: 0.1459 - acc: 0.9347 - val_loss: 0.1644 - val_acc: 0.9307
Epoch 6/15
27234/27234 [==============================] - 427s - loss: 0.1250 - acc: 0.9439 - val_loss: 0.1808 - val_acc: 0.9289
Epoch 7/15
27234/27234 [==============================] - 434s - loss: 0.1227 - acc: 0.9442 - val_loss: 0.2058 - val_acc: 0.9268
Epoch 8/15
27234/27234 [===============

In [78]:
score, acc = model.evaluate(test, y_test,
                            batch_size=50)
print('Test score:', score)
print('Test accuracy:', acc)

2683/2683 [==============================] - 12s    
('Test score:', 0.20978614839125026)
('Test accuracy:', 0.92396571182555953)


## Try with GRU and increased hiddedn layers

In [22]:
print('Build model_gru...')
model_gru = Sequential()
model_gru.add(Embedding(len(vocab)+1, 128, dropout=0.2, mask_zero=True))
model_gru.add(GRU(300, dropout_W=0.2, dropout_U=0.2))  # try using a GRU instead, for fun
model_gru.add(Dense(1))
model_gru.add(Activation('sigmoid'))

# try using different optimizers and different optimizer configs
model_gru.compile(loss='binary_crossentropy',
              optimizer='adagrad',
              metrics=['accuracy'])

Build model_gru...


In [23]:
print('Train...')
model_gru.fit(train, y_train, batch_size=50, nb_epoch=15,validation_data=(val, y_val))

Train...
Train on 27235 samples, validate on 6765 samples
Epoch 1/15
27235/27235 [==============================] - 631s - loss: 0.3687 - acc: 0.8344 - val_loss: 0.2446 - val_acc: 0.8918
Epoch 2/15
27235/27235 [==============================] - 614s - loss: 0.2481 - acc: 0.8898 - val_loss: 0.2247 - val_acc: 0.8990
Epoch 3/15
27235/27235 [==============================] - 608s - loss: 0.2111 - acc: 0.9067 - val_loss: 0.2161 - val_acc: 0.9042
Epoch 4/15
27235/27235 [==============================] - 606s - loss: 0.1878 - acc: 0.9180 - val_loss: 0.2167 - val_acc: 0.9041
Epoch 5/15
27235/27235 [==============================] - 607s - loss: 0.1690 - acc: 0.9259 - val_loss: 0.2178 - val_acc: 0.9026
Epoch 6/15
27235/27235 [==============================] - 603s - loss: 0.1562 - acc: 0.9315 - val_loss: 0.2213 - val_acc: 0.9036
Epoch 7/15
27235/27235 [==============================] - 607s - loss: 0.1407 - acc: 0.9354 - val_loss: 0.2300 - val_acc: 0.9011
Epoch 8/15
27235/27235 [===============

In [24]:
score, acc = model_gru.evaluate(test, y_test,
                            batch_size=50)
print('Test score:', score)
print('Test accuracy:', acc)

4200/4200 [==============================] - 27s    
('Test score:', 0.28316650934339987)
('Test accuracy:', 0.90428570977279121)


## =================================================
## Try categorial classification 

In [19]:
print('Build model_cat...')
model_cat = Sequential()
model_cat.add(Embedding(len(vocab)+1, 128, dropout=0.2, mask_zero=True))
model_cat.add(LSTM(128, dropout_W=0.2, dropout_U=0.2))  # try using a GRU instead, for fun
model_cat.add(Dense(5))
model_cat.add(Activation('sigmoid'))

# try using different optimizers and different optimizer configs
model_cat.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', 'mean_absolute_error'])

Build model_cat...


In [20]:
print('Train...')

model_cat.fit(train, y_cat_train, batch_size=50, nb_epoch=15,validation_data=(val, y_cat_val))

Train...


/usr/lib/python2.7/site-packages/tensorflow/python/ops/gradients.py:90: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 27235 samples, validate on 6765 samples
Epoch 1/15
27235/27235 [==============================] - 431s - loss: 0.9502 - acc: 0.6774 - mean_absolute_error: 0.2114 - val_loss: 0.7005 - val_acc: 0.7786 - val_mean_absolute_error: 0.1381
Epoch 2/15
27235/27235 [==============================] - 426s - loss: 0.6953 - acc: 0.7762 - mean_absolute_error: 0.1429 - val_loss: 0.6500 - val_acc: 0.7899 - val_mean_absolute_error: 0.1294
Epoch 3/15
27235/27235 [==============================] - 411s - loss: 0.6179 - acc: 0.7947 - mean_absolute_error: 0.1303 - val_loss: 0.6340 - val_acc: 0.7910 - val_mean_absolute_error: 0.1258
Epoch 4/15
27235/27235 [==============================] - 414s - loss: 0.5702 - acc: 0.8041 - mean_absolute_error: 0.1277 - val_loss: 0.6351 - val_acc: 0.7931 - val_mean_absolute_error: 0.1268
Epoch 5/15
27235/27235 [==============================] - 407s - loss: 0.5216 - acc: 0.8174 - mean_absolute_error: 0.1263 - val_loss: 0.6400 - val_acc: 0.7830 - val_mean_absolute_

In [21]:
score, acc, mae = model_cat.evaluate(test, y_cat_test,
                            batch_size=50)
print('Test score:', score)
print('Test accuracy:', acc)
print('Test Mean absolute error:', mae)

4200/4200 [==============================] - 19s    
('Test score:', 0.84444178853716167)
('Test accuracy:', 0.77642857247874852)
('Test Mean absolute error:', 0.13243585567744004)


##  ================================================
## Add word2vec

In [74]:
print('Build model_w2v...')
model_w2v = Sequential()
model_w2v.add(Embedding(len(W), 300, dropout=0.2, weights=[W], mask_zero=True))
model_w2v.add(LSTM(128, dropout_W=0.3, dropout_U=0.3))  # try using a GRU instead, for fun
model_w2v.add(Dense(1))
model_w2v.add(Activation('tanh'))

# try using different optimizers and different optimizer configs
model_w2v.compile(loss='binary_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

Build model_w2v...


In [75]:
print('Train...')
model_w2v.fit(train, y_train, batch_size=50, nb_epoch=15,validation_data=(val, y_val))

Train...
Train on 27235 samples, validate on 6765 samples
Epoch 1/15
27235/27235 [==============================] - 603s - loss: 0.7838 - acc: 0.6541 - val_loss: 0.4998 - val_acc: 0.7438
Epoch 2/15
27235/27235 [==============================] - 598s - loss: 0.4971 - acc: 0.7786 - val_loss: 0.3700 - val_acc: 0.8177
Epoch 3/15
27235/27235 [==============================] - 597s - loss: 0.4282 - acc: 0.8154 - val_loss: 0.3512 - val_acc: 0.8636
Epoch 4/15
27235/27235 [==============================] - 597s - loss: 0.3906 - acc: 0.8356 - val_loss: 0.3466 - val_acc: 0.8680
Epoch 5/15
27235/27235 [==============================] - 593s - loss: 0.3712 - acc: 0.8459 - val_loss: 0.2822 - val_acc: 0.8806
Epoch 6/15
27235/27235 [==============================] - 586s - loss: 0.3603 - acc: 0.8554 - val_loss: 0.2735 - val_acc: 0.8838
Epoch 7/15
27235/27235 [==============================] - 589s - loss: 0.3411 - acc: 0.8591 - val_loss: 0.2696 - val_acc: 0.8891
Epoch 8/15
27235/27235 [===============

In [76]:
score, acc = model_w2v.evaluate(test, y_test,
                            batch_size=50)
print('Test score:', score)
print('Test accuracy:', acc)

4200/4200 [==============================] - 24s    
('Test score:', 0.29426618505801472)
('Test accuracy:', 0.89928571099326726)
